# Diseño de Sistemas de Adquisición y Procesamiento Masivo de Datos

## Práctica 1

### Yago Boleas y Zakaria Lasry

In [ ]:
import os
import plotly.graph_objs as go
import polars as pl
from sklearn.cluster import DBSCAN
from pathlib import Path
import numpy as np
from tqdm import tqdm

In [ ]:
data = dict(
    carretera=Path("./data/0_carretera.csv"),
    portico=Path("./data/1_portico.csv"),
    coche=Path("./data/2_coche.csv"),
    coches=Path("./data/3_coche_coche.csv"),
    coches_moto=Path("./data/4_coche_coche_moto.csv"),
    video=Path("./data/5_video_nube_de_puntos"),
)

min_, max_ = -30, 30

In [ ]:
def clean_lidar_points(
    df: pl.DataFrame, iqr_factor: float = 2.0, verbose: bool = True
) -> pl.DataFrame:
    df_nodup = df.unique(subset=["x", "y", "z"])

    q = df_nodup.select(
        [
            pl.col("x").quantile(0.25).alias("x_q25"),
            pl.col("x").quantile(0.75).alias("x_q75"),
            pl.col("y").quantile(0.25).alias("y_q25"),
            pl.col("y").quantile(0.75).alias("y_q75"),
            pl.col("z").quantile(0.25).alias("z_q25"),
            pl.col("z").quantile(0.75).alias("z_q75"),
        ]
    ).to_dicts()[0]

    x_iqr = q["x_q75"] - q["x_q25"]
    y_iqr = q["y_q75"] - q["y_q25"]
    z_iqr = q["z_q75"] - q["z_q25"]

    x_min, x_max = q["x_q25"] - iqr_factor * x_iqr, q["x_q75"] + iqr_factor * x_iqr
    y_min, y_max = q["y_q25"] - iqr_factor * y_iqr, q["y_q75"] + iqr_factor * y_iqr
    z_min, z_max = q["z_q25"] - iqr_factor * z_iqr, q["z_q75"] + iqr_factor * z_iqr

    clean_df = df_nodup.filter(
        (pl.col("x").is_between(x_min, x_max))
        & (pl.col("y").is_between(y_min, y_max))
        & (pl.col("z").is_between(z_min, z_max))
    )

    if verbose:
        print(f"Puntos originales: {df.height}")
        print(f"Puntos tras eliminar duplicados y outliers: {clean_df.height}")

    return clean_df


def plot_scene(
    df, color_var, dataset=None, pov=False, colorscale="hot", opacity=0.8, fig=None
):
    if fig is None:
        fig = go.Figure(
            layout=go.Layout(
                width=800,
                height=800,
                scene=dict(
                    xaxis=dict(title="-X", range=[min_, max_]),
                    yaxis=dict(title="Z", range=[min_, max_]),
                    zaxis=dict(title="-Y", range=[min_, max_]),
                ),
            ),
        )

    fig.add_trace(
        go.Scatter3d(
            x=-df["x"],
            y=df["z"],
            z=-df["y"],
            mode="markers",
            name=color_var,
            marker=dict(
                size=1,
                color=df[color_var],
                colorscale=colorscale,
                opacity=opacity,
                colorbar=dict(title=color_var),
            ),
            hoverinfo="text",
            hovertext=df[color_var],
        )
    )

    if pov:
        fig.add_trace(
            go.Scatter3d(
                x=[0],
                y=[0],
                z=[0],
                mode="markers",
                name="Lidar POV",
                marker=dict(size=5, color="Red"),
                hoverinfo="text",
                hovertext="Lidar POV",
            )
        )

    if dataset or fig.layout.title.text == "":
        fig.update_layout(
            showlegend=True,
            title=dict(
                text=f"Lidar Point Cloud{f': {dataset}' if dataset else ''}",
                x=0.5,
                y=0.9,
                xanchor="center",
                yanchor="top",
                font=dict(
                    family="Arial, monospace",
                    size=32,
                    color="Black",
                    variant="small-caps",
                ),
            ),
            font=dict(
                family="Arial, monospace",
                size=12,
                color="Black",
                variant="small-caps",
            ),
        )
    fig.show()
    return fig

In [ ]:
def create_bounding_box_lines(min_x, max_x, min_y, max_y, min_z, max_z):
    x_lines, y_lines, z_lines = [], [], []
    # The 8 corners of the box
    corners = [
        (min_x, min_y, min_z),
        (min_x, min_y, max_z),
        (min_x, max_y, min_z),
        (min_x, max_y, max_z),
        (max_x, min_y, min_z),
        (max_x, min_y, max_z),
        (max_x, max_y, min_z),
        (max_x, max_y, max_z),
    ]
    # The 12 lines connecting the corners
    edges = [
        (0, 1),
        (0, 2),
        (0, 4),
        (1, 3),
        (1, 5),
        (2, 3),
        (2, 6),
        (3, 7),
        (4, 5),
        (4, 6),
        (5, 7),
        (6, 7),
    ]
    for start, end in edges:
        x_lines.extend([corners[start][0], corners[end][0], None])
        y_lines.extend([corners[start][1], corners[end][1], None])
        z_lines.extend([corners[start][2], corners[end][2], None])
    return x_lines, y_lines, z_lines

## ANÁLISIS DE LOS DATOS

Lo primero es echar un vistazo a las distintas variables que hay en los conjuntos de datos. Usamos el primer conjunto de datos `carretera` para esto

In [ ]:
df = pl.read_csv(data["carretera"])
df = df.unique(["x", "y", "z"])
df

### Descripción de variables

- **x**: coordenada cartesiana en el eje horizontal (eje X).
- **y**: coordenada cartesiana en el eje horizontal (eje Y).
- **z**: coordenada cartesiana en el eje vertical (eje Z).
- **intensity**: intensidad de la señal reflejada que regresa al sensor. Indica cuánta luz láser es reflejada por el objeto detectado.
- **t**: marca de tiempo (*timestamp*) en la que se tomó la medición.
- **reflectivity**: reflectividad del objeto detectado. Mide la capacidad del objeto para reflejar la luz láser.
- **ring**: índice del anillo o línea de escaneo al que pertenece el punto.
- **ambient**: nivel de luz ambiental presente en el entorno. 
- **range**: distancia desde el sensor Lidar hasta el objeto detectado, medida en metros.

### Primer vistazo a los datos

En esta figura podemos ver la carretera sin ningún tipo de obtáculo por en medio. Se ha decidido también representar la variable `range`, que representa la distancia a la que se encuentra cada uno de los puntos al sensor Lidar. Se muestra también el punto (0, 0, 0) para poder apreciar con mayor claridad esta variable.

Para representar los datos también se ha realizado un pequeño procesado de los datos, compuesto por dos etapas:

1. **Eliminación de duplicados**: se eliminaron los puntos que tenían coordenadas `x`, `y` y `z` idénticas.
2. **Filtrado de puntos atípicos**: se aplicó un método estadístico basado en el *Rango Intercuartílico (IQR)* para detectar y eliminar los puntos que estaban demasiado lejos del grupo principal. Específicamente, se eliminaron todos aquellos puntos que se encontraban a una distancia mayor de 2 veces el IQR por debajo del cuartil 25 o por encima del cuartil 75 para cada eje `(x, y, z)`.

Además de la limpieza, se ha representado la variable `range`, que indica la distancia de cada punto al sensor LiDAR, utilizando una **escala de colores** ('*hot*'). Para facilitar la comprensión espacial, se ha añadido un punto de referencia en el origen `(0, 0, 0)`. Es importante notar que, para la visualización, se han ajustado los ejes para una mejor perspectiva: el *eje Z* del gráfico representa la coordenada `y` del sensor (invertida) y el *eje Y* del gráfico la coordenada `z`.  Finalmente, los rangos de los ejes se han limitado a `[-30, 30]` para centrar la vista en el área de interés.

In [ ]:
dataset = "carretera"
color_var = "range"
df = pl.read_csv(data[dataset])
df = clean_lidar_points(df, verbose=False)
fig = plot_scene(df, color_var, dataset, pov=True)

### Algoritmo de eliminación de puntos estáticos

En esta nueva escena se emplean de nuevo los datos de la carretera vacía. El objetivo de esta parte es obtener una representación de las zonas estáticas dentro de la visión del Lidar. Para ello, se implementa un método de **voxelización** para crear un modelo estático del entorno. Cuando llegue una nueva escena, se comprobará que los datos pertenezcan o no a una de las distintas regiones. En caso de pertenecer a esta zona no se tendrán en cuenta a la hora de hacer la clusterización. El proceso para obtener esa escena estática es el siguiente:

1.  **Voxelización del espacio**: se define una región de interés en el espacio tridimensional (un cubo de $60 \times 60 \times 60$ metros, de -30 a 30 en cada eje) y se divide en una rejilla de $120 \times 120 \times 120$ voxels (un voxel es el equivalente tridimensional de un píxel).
2.  **Asignación de puntos a voxels**: cada punto de la nube de puntos se asigna a un voxel específico. Esto se logra calculando las coordenadas del voxel (`vx`, `vy`, `vz`) para cada punto basándose en sus coordenadas `x`, `y` y `z` y el tamaño del voxel.
3.  **Conteo de puntos por voxel**: se cuenta el número de puntos de la nube que caen dentro de cada voxel. Este conteo se almacena en el array tridimensional `voxel`. Un voxel con un conteo mayor a cero indica que esa región del espacio está "ocupada" por un objeto estático.

Para dar una idea del resultado de la voxelización se muestra una figura tridimensional con todos los voxels hallados, donde:

-   El **tamaño** de los marcadores (cada uno de los puntos) en el gráfico está escalado en función del tamaño del voxel, lo que da una idea de la granularidad de la rejilla.
-   El **color** de cada marcador representa el **conteo de puntos** en ese voxel. Esto permite identificar áreas con mayor densidad de puntos, como las superficies de la carretera o la estructura del pórtico.
-   Para una visualización más intuitiva, los ejes del gráfico se han intercambiado (`x` $\Rightarrow$ `-vx`, `y` $\Rightarrow$ `vz`, `z` $\Rightarrow$ `-vy`) para que el pórtico se muestre en una orientación más convencional.

Este modelo de voxels sirve como una "huella" de la escena estática. En el futuro, cuando llegue una nueva nube de puntos, cualquier punto que no caiga en un voxel previamente ocupado se considera parte de un **objeto móvil**, lo que facilita su detección y aislamiento.

In [ ]:
def make_voxel_grid(
    df: pl.DataFrame,
    n_regions: int = 120,
    min_value: float = min_,
    max_value: float = max_,
) -> pl.DataFrame:
    voxel_size_x = (max_value - min_value) / n_regions
    voxel_size_y = (max_value - min_value) / n_regions
    voxel_size_z = (max_value - min_value) / n_regions

    static_pts = df.with_columns(
        (((pl.col("x") - min_value) / voxel_size_x).floor().clip(0, n_regions - 1))
        .cast(pl.Int32)
        .alias("vx"),
        (((pl.col("y") - min_value) / voxel_size_y).floor().clip(0, n_regions - 1))
        .cast(pl.Int32)
        .alias("vy"),
        (((pl.col("z") - min_value) / voxel_size_z).floor().clip(0, n_regions - 1))
        .cast(pl.Int32)
        .alias("vz"),
    )
    count = static_pts.group_by(["vx", "vy", "vz"]).len().rename({"len": "count"})
    # voxel = np.zeros((n_regions, n_regions, n_regions), dtype=np.uint16)
    # voxel[count["vx"], count["vy"], count["vz"]] = count["count"]
    return count


def mark_static_points(
    df_scene: pl.DataFrame,
    static_voxels: pl.DataFrame,
    n_regions: int,
    min_value: float = min_,
    max_value: float = max_,
) -> pl.DataFrame:
    voxel_size_x = (max_value - min_value) / n_regions
    voxel_size_y = (max_value - min_value) / n_regions
    voxel_size_z = (max_value - min_value) / n_regions

    df_scene = df_scene.with_columns(
        (((pl.col("x") - min_value) / voxel_size_x).floor().clip(0, n_regions - 1))
        .cast(pl.Int32)
        .alias("vx"),
        (((pl.col("y") - min_value) / voxel_size_y).floor().clip(0, n_regions - 1))
        .cast(pl.Int32)
        .alias("vy"),
        (((pl.col("z") - min_value) / voxel_size_z).floor().clip(0, n_regions - 1))
        .cast(pl.Int32)
        .alias("vz"),
    )

    static_flag = static_voxels.with_columns(pl.lit(-2).alias("cluster"))

    df_scene = df_scene.join(
        static_flag.select(["vx", "vy", "vz", "cluster"]),
        on=["vx", "vy", "vz"],
        how="left",
    ).fill_null(pl.lit(-1))

    return df_scene

In [ ]:
static_pts = pl.read_csv(data["carretera"])
static_pts = static_pts.unique(subset=["x", "y", "z"])
regions = 240
voxels = make_voxel_grid(static_pts, n_regions=regions)

fig = go.Figure(
    data=[
        go.Scatter3d(
            x=-voxels["vx"],
            y=voxels["vz"],
            z=-voxels["vy"],
            mode="markers",
            marker=dict(
                size=5 * ((max_ - min_) / regions),
                color=voxels["count"],
                colorbar=dict(
                    title="count",
                    tickfont=dict(size=12),
                    title_font=dict(size=14),
                ),
                opacity=0.8,
            ),
            hoverinfo="text",
            hovertext=voxels["count"],
        )
    ],
    layout=go.Layout(
        width=800,
        height=800,
        scene=dict(
            xaxis=dict(range=[-regions, 0]),
            yaxis=dict(range=[0, regions]),
            zaxis=dict(range=[-regions, 0]),
        ),
    ),
)

fig.show()

---

In [ ]:
def clusters_with_dbscan(
    df: pl.DataFrame,
    feature_cols: list[str],
    eps: float = 0.5,
    min_samples: int = 10,
) -> pl.DataFrame:
    df = df.with_row_index(name="id")
    subset = df.filter(pl.col("cluster") != -2)
    labels = DBSCAN(eps=eps, min_samples=min_samples).fit_predict(
        subset.select(feature_cols).to_numpy()
    )
    subset = subset.with_columns(pl.Series("cluster", labels))
    df = (
        df.join(subset.select(["id", "cluster"]), on="id", how="left")
        .with_columns(
            pl.when(pl.col("cluster_right").is_not_null())
            .then(pl.col("cluster_right"))
            .otherwise(pl.col("cluster"))
            .alias("cluster")
        )
        .drop("cluster_right")
    )
    return df

In [ ]:
color_var = "cluster"
dataset = "portico"

df2 = pl.read_csv(data[dataset])
df2 = clean_lidar_points(df2, verbose=False)
df2 = mark_static_points(df2, voxels, n_regions=regions)
df2 = clusters_with_dbscan(
    df2,
    ["x", "y", "z"],
)
print(df2["cluster"])

fig = plot_scene(df2, color_var, dataset, pov=False, opacity=1)

---

In [ ]:
color_var = "cluster"
dataset = "coche"

df3 = pl.read_csv(data[dataset])
df3 = clean_lidar_points(df3, verbose=False)
df3 = mark_static_points(df3, voxels, n_regions=regions)
df3 = clusters_with_dbscan(
    df3,
    ["x", "y", "z"],
)
fig = plot_scene(df3, color_var, dataset, pov=True, opacity=1)
for xmin, xmax, ymin, ymax, zmin, zmax in cluster_bounding_cubes(df3):
    fig = add_cube(fig, xmin, xmax, ymin, ymax, zmin, zmax)

---

In [ ]:
color_var = "cluster"
dataset = "coches"

df4 = pl.read_csv(data[dataset])
df4 = clean_lidar_points(df4, verbose=False)
df4 = mark_static_points(df4, voxels, n_regions=regions)
df4 = clusters_with_dbscan(
    df4,
    ["x", "y", "z"],
)
plot_scene(df4, color_var, dataset, pov=True, opacity=1)

---

In [ ]:
color_var = "cluster"
dataset = "coches_moto"

df5 = pl.read_csv(data[dataset])
df5 = clean_lidar_points(df5, verbose=False)
df5 = mark_static_points(df5, voxels, n_regions=regions)
df5 = clusters_with_dbscan(
    df5,
    ["x", "y", "z"],
)
plot_scene(df5.filter(pl.col("cluster") >= -1), color_var, dataset, pov=True, opacity=1)

---

In [ ]:
lazy_dfs = [
    pl.scan_csv(os.path.join(data["video"], file)).with_columns(
        pl.lit(i).alias("frame")
    )
    for i, file in enumerate(sorted(os.listdir(data["video"])))
]
df_raw = pl.concat(lazy_dfs).collect()

frames = df_raw["frame"].unique().sort().to_list()

In [ ]:
from scipy.spatial.distance import cdist


def compute_special_centroids(points: np.ndarray, labels: np.ndarray) -> dict:
    """
    Calcula el 'centroide especial' de cada cluster, definido como
    el centro geométrico de la bounding box de sus puntos.
    Devuelve un dict {cluster_id: (cx, cy)}.
    """
    centroids = {}
    for cluster_id in np.unique(labels):
        if cluster_id == -1:
            continue  # Ignorar ruido
        cluster_points = points[labels == cluster_id]
        min_x, min_y = cluster_points[:, 0].min(), cluster_points[:, 1].min()
        max_x, max_y = cluster_points[:, 0].max(), cluster_points[:, 1].max()
        cx, cy = (min_x + max_x) / 2, (min_y + max_y) / 2
        centroids[cluster_id] = np.array([cx, cy])
    return centroids


def match_clusters(
    prev_centroids: dict, new_centroids: dict, threshold: float = 1.0
) -> dict:
    """
    Asigna los nuevos centroides a los antiguos en función de la distancia mínima.
    Devuelve un dict {new_cluster_id: matched_prev_id} (o None si no hay match cercano).
    """
    matches = {}
    if not prev_centroids or not new_centroids:
        return matches

    prev_ids = list(prev_centroids.keys())
    new_ids = list(new_centroids.keys())
    prev_points = np.array([prev_centroids[i] for i in prev_ids])
    new_points = np.array([new_centroids[i] for i in new_ids])

    dist_matrix = cdist(new_points, prev_points)
    for i, new_id in enumerate(new_ids):
        min_idx = np.argmin(dist_matrix[i])
        if dist_matrix[i, min_idx] <= threshold:
            matches[new_id] = prev_ids[min_idx]
        else:
            matches[new_id] = None
    return matches


def compute_velocities(
    prev_centroids: dict, new_centroids: dict, matches: dict, dt: float = 1.0
) -> dict:
    """
    Calcula las velocidades (vx, vy) entre centroides emparejados.
    Devuelve {cluster_id: (vx, vy)}.
    """
    velocities = {}
    for new_id, prev_id in matches.items():
        if prev_id is None:
            continue
        prev_pos = prev_centroids[prev_id]
        new_pos = new_centroids[new_id]
        vx, vy = (new_pos - prev_pos) / dt
        velocities[new_id] = np.array([vx, vy])
    return velocities


def predict_positions(
    current_centroids: dict, velocities: dict, dt: float = 1.0
) -> dict:
    """
    Predice la posición esperada de cada cluster usando su velocidad.
    Devuelve {cluster_id: (pred_x, pred_y)}.
    """
    preds = {}
    for cid, pos in current_centroids.items():
        if cid in velocities:
            preds[cid] = pos + velocities[cid] * dt
        else:
            preds[cid] = pos
    return preds


def update_tracking_state(
    state_df: pl.DataFrame,
    frame_id: int,
    centroids: dict,
    velocities: dict,
    preds: dict,
):
    """
    Actualiza un DataFrame de estado de tracking con las posiciones, velocidades y predicciones.
    """
    rows = []
    for cid in centroids:
        cx, cy = centroids[cid]
        vx, vy = velocities.get(cid, (np.nan, np.nan))
        px, py = preds.get(cid, (np.nan, np.nan))
        rows.append((frame_id, cid, cx, cy, vx, vy, px, py))
    new_df = pl.DataFrame(
        rows,
        schema=["frame", "cluster", "cx", "cy", "vx", "vy", "px", "py"],
        orient="row",
    )

    return pl.concat([state_df, new_df])


# ============================================================
# =============== PIPELINE PRINCIPAL =========================
# ============================================================

dbscan = DBSCAN(eps=2, min_samples=20)
processed_dfs = []
tracking_state = pl.DataFrame(
    {
        "frame": pl.Series([], dtype=pl.Int64),
        "cluster": pl.Series([], dtype=pl.Int64),
        "cx": pl.Series([], dtype=pl.Float64),
        "cy": pl.Series([], dtype=pl.Float64),
        "vx": pl.Series([], dtype=pl.Float64),
        "vy": pl.Series([], dtype=pl.Float64),
        "px": pl.Series([], dtype=pl.Float64),
        "py": pl.Series([], dtype=pl.Float64),
    }
)

frames_to_use = frames[:10]

# --- FRAME 0 ---
frame0 = df_raw.filter(pl.col("frame") == frames_to_use[0])
points0 = frame0.select(["x", "y", "z"]).to_numpy()
labels0 = dbscan.fit_predict(points0)
centroids0 = compute_special_centroids(points0, labels0)
processed_dfs.append(frame0.with_columns(pl.Series("cluster", labels0)))

# --- FRAME 1 ---
frame1 = df_raw.filter(pl.col("frame") == frames_to_use[1])
points1 = frame1.select(["x", "y", "z"]).to_numpy()
labels1 = dbscan.fit_predict(points1)
centroids1 = compute_special_centroids(points1, labels1)
matches01 = match_clusters(centroids0, centroids1)
velocities1 = compute_velocities(centroids0, centroids1, matches01)
preds1 = predict_positions(centroids1, velocities1)

tracking_state = update_tracking_state(
    tracking_state, frames_to_use[1], centroids1, velocities1, preds1
)
processed_dfs.append(frame1.with_columns(pl.Series("cluster", labels1)))

# --- RESTO DE FRAMES ---
prev_centroids = centroids1
prev_velocities = velocities1
prev_preds = preds1

for frame_id in tqdm(frames_to_use[2:], desc="Tracking vehicles"):
    frame_df = df_raw.filter(pl.col("frame") == frame_id)
    points = frame_df.select(["x", "y", "z"]).to_numpy()

    labels = dbscan.fit_predict(points)
    centroids = compute_special_centroids(points, labels)

    matches = match_clusters(prev_preds, centroids, threshold=1.0)
    velocities = compute_velocities(prev_centroids, centroids, matches)
    preds = predict_positions(centroids, velocities)

    tracking_state = update_tracking_state(
        tracking_state, frame_id, centroids, velocities, preds
    )
    processed_dfs.append(frame_df.with_columns(pl.Series("cluster", labels)))

    prev_centroids = centroids
    prev_velocities = velocities
    prev_preds = preds

In [ ]:
import plotly.express as px

def plot_lidar_clusters_3d_interactive(df: pl.DataFrame):
    """
    Muestra un scatter 3D interactivo de los puntos LiDAR coloreados por cluster,
    con un slider para moverse entre frames.

    Parámetros:
        df: polars.DataFrame
            DataFrame concatenado con columnas ['x','y','z','cluster','frame'].
    """
    # Aseguramos que las columnas necesarias existen
    required_cols = {"x", "y", "z", "cluster", "frame"}
    if not required_cols.issubset(df.columns):
        raise ValueError(f"El DataFrame debe contener las columnas {required_cols}")


    # Scatter 3D animado con slider
    fig = px.scatter_3d(
        df,
        x="x",
        y="y",
        z="z",
        color="cluster",
        animation_frame="frame",
        title="Evolución temporal de los clusters LiDAR",
        width=900,
        height=700,
        color_continuous_scale="Viridis"  # escala de color agradable
    )

    fig.update_traces(marker=dict(size=3, opacity=0.8))
    fig.update_layout(
        scene=dict(
            xaxis_title="X",
            yaxis_title="Y",
            zaxis_title="Z",
            aspectmode="data"  # mantiene proporciones reales
        ),
        margin=dict(l=0, r=0, b=0, t=40),
        coloraxis_colorbar=dict(title="Cluster"),
    )

    fig.show()

plot_lidar_clusters_3d_interactive(pl.concat(processed_dfs))

In [ ]:
processed_dfs = []

dbscan = DBSCAN(eps=2, min_samples=20)

print("Clustering points for each frame...")
for frame_id in frames:
    frame_df = df_raw.filter(pl.col("frame") == frame_id)
    points = frame_df.select(["x", "y", "z"]).to_numpy()

    if len(points) > 0:
        clusters = dbscan.fit_predict(points)
        processed_df = frame_df.with_columns(pl.Series("cluster", clusters))
        processed_dfs.append(processed_df)

df_clustered = pl.concat(processed_dfs)

print("Clustering complete.")

all_values = pl.concat([df_clustered["x"], df_clustered["y"], df_clustered["z"]])
min_global = all_values.min()
max_global = all_values.max()

animation_frames = []
for f in frames:
    current_frame_data = df_clustered.filter(pl.col("frame") == f)

    non_noise_points = current_frame_data.filter(pl.col("cluster") >= 0)

    cluster_labels = non_noise_points["cluster"].unique().to_list()

    box_x, box_y, box_z = [], [], []
    for cluster_id in cluster_labels:
        cluster_points = current_frame_data.filter(pl.col("cluster") == cluster_id)

        min_vals = cluster_points.select(["x", "y", "z"]).min().row(0)
        max_vals = cluster_points.select(["x", "y", "z"]).max().row(0)

        x_lines, y_lines, z_lines = create_bounding_box_lines(
            min_vals[0], max_vals[0], min_vals[1], max_vals[1], min_vals[2], max_vals[2]
        )
        box_x.extend(x_lines)
        box_y.extend(y_lines)
        box_z.extend(z_lines)

    current_frame_data = df_clustered.filter(pl.col("frame") == f)

    cluster_labels = current_frame_data["cluster"]
    cluster_labels = cluster_labels.filter(cluster_labels >= 0).unique().to_list()

    box_x, box_y, box_z = [], [], []
    for cluster_id in cluster_labels:
        cluster_points = current_frame_data.filter(pl.col("cluster") == cluster_id)

        min_vals = cluster_points.select(["x", "y", "z"]).min().row(0)
        max_vals = cluster_points.select(["x", "y", "z"]).max().row(0)

        x_lines, y_lines, z_lines = create_bounding_box_lines(
            min_vals[0], max_vals[0], min_vals[1], max_vals[1], min_vals[2], max_vals[2]
        )
        box_x.extend(x_lines)
        box_y.extend(y_lines)
        box_z.extend(z_lines)

    animation_frames.append(
        go.Frame(
            data=[
                go.Scatter3d(
                    x=current_frame_data["x"],
                    y=current_frame_data["y"],
                    z=current_frame_data["z"],
                    mode="markers",
                    marker=dict(size=1, color="red", opacity=0.8),
                ),
                go.Scatter3d(
                    x=box_x,
                    y=box_y,
                    z=box_z,
                    mode="lines",
                    line=dict(color="black", width=2),
                ),
            ],
            name=str(f),
        )
    )


df0 = animation_frames[0].data

fig = go.Figure(
    data=df0,
    layout=go.Layout(
        width=800,
        height=800,
        scene=dict(
            xaxis=dict(title="X", range=[min_global, max_global]),
            yaxis=dict(title="Y", range=[min_global, max_global]),
            zaxis=dict(title="Z", range=[min_global, max_global]),
            aspectmode="cube",
        ),
    ),
    frames=animation_frames,
)

fig.update_layout(
    title=dict(
        text="Lidar Point Cloud: video",
        x=0.5,
        y=0.9,
        xanchor="center",
        yanchor="top",
        font=dict(
            family="Arial, monospace", size=32, color="Black", variant="small-caps"
        ),
    ),
    font=dict(
        family="Arial, monospace",
        size=12,
        color="Black",
        variant="small-caps",
    ),
    sliders=[
        dict(
            steps=[
                dict(
                    args=[
                        [str(f)],
                        dict(
                            frame={"duration": 200, "redraw": True},
                            mode="immediate",
                        ),
                    ],
                    label=str(f),
                    method="animate",
                )
                for f in frames
            ],
            transition={"duration": 0},
            x=0.1,
            y=0,
            len=0.9,
        )
    ],
)


fig.show()

In [ ]:
temp = df_raw.filter(pl.col("frame") == 200)
labels = DBSCAN(eps=2, min_samples=20).fit_predict(temp.select(["x", "z"]))
temp = temp.with_columns(pl.Series("cluster", labels))

centroids = (
    temp.filter(pl.col("cluster") != -1)
    .group_by("cluster")
    .agg(
        [
            pl.col("x").mean(),
            pl.col("y").mean(),
            pl.col("z").mean(),
        ]
    )
    .sort("cluster")
    .with_columns(pl.lit(True).alias("active"))
)

centroids
temp2 = df_raw.filter(pl.col("frame") == 201)
labels = DBSCAN(eps=2, min_samples=20).fit_predict(temp2.select(["x", "z"]))
temp2 = temp2.with_columns(pl.Series("cluster", labels))
centroids2 = (
    temp2.filter(pl.col("cluster") != -1)
    .group_by("cluster")
    .agg(
        [
            pl.col("x").mean(),
            pl.col("y").mean(),
            pl.col("z").mean(),
        ]
    )
    .sort("cluster")
    .with_columns(pl.lit(True).alias("active"))
)

print(centroids, centroids2)

# c1 = centroids.select(['x', 'y', 'z']).to_numpy()
# c2 = centroids2.select(['x', 'y', 'z']).to_numpy()
# np.linalg.norm(c2-c1, axis=1)

In [ ]:
c1 = centroids.select(["x", "y", "z"]).to_numpy()
c2 = centroids2[0].select(["x", "y", "z"]).to_numpy()
(np.linalg.norm(c2 - c1, axis=1))
centroids[:, :4]

In [ ]:
def get_centroids(df: pl.DataFrame):
    return (
        df.filter(pl.col("cluster") != -1)
        .group_by("cluster")
        .agg([pl.col("x").mean(), pl.col("y").mean(), pl.col("z").mean()])
        .sort("cluster")
        .with_columns(pl.lit(True).alias("active"))
    )

In [ ]:
centroids = pl.DataFrame({"x": [], "y": [], "z": [], "active": []})
centroids.height

In [ ]:
THRESHOLD = 1.0
all_centroids = pl.DataFrame({"x": [], "y": [], "z": [], "active": [], "frame": []})
for frame in tqdm(df_raw["frame"].unique().to_list()):
    temp = df_raw.filter(pl.col("frame") == frame)
    labels = DBSCAN(eps=2, min_samples=20).fit_predict(temp.select(["x", "y", "z"]))
    temp = temp.with_columns(pl.Series("cluster", labels))
    new_centroids = get_centroids(temp)
    new_centroids = new_centroids.with_columns(pl.lit(frame).alias("frame"))
    if all_centroids.height == 0:
        all_centroids = new_centroids
    else:
        for cent in new_centroids.select(["x", "y", "z"]).iter_rows():
            centroids = all_centroids.select(["x", "y", "z"]).to_numpy()
            cent = np.array(cent)
            dist = np.linalg.norm(centroids - cent, axis=1)
            idx = int(np.argmin(dist))
            if dist[0] <= THRESHOLD:
                # print(all_centroids[idx, :4])
                all_centroids[idx, "x"] = cent[0]
                all_centroids[idx, "y"] = cent[1]
                all_centroids[idx, "z"] = cent[2]
            else:
                all_centroids = pl.concat(
                    [
                        all_centroids,
                        pl.DataFrame(
                            {
                                "cluster": [all_centroids.height],
                                "x": [cent[0]],
                                "y": [cent[1]],
                                "z": [cent[2]],
                                "active": [True],
                            }
                        ),
                    ]
                )

In [ ]:
all_centroids.head(10)

In [ ]:
np.linalg.norm(np.array([-17.35, 1.84, 1.18]) - np.array([-17.51, 1.85, 1.17]))

In [ ]:
temp = df_raw.filter(pl.col("frame") == 123)
temp = clusters_with_dbscan(temp, ["x", "y", "z"], eps=2, min_samples=20)


fig = go.Figure(
    data=[
        go.Scatter3d(
            x=temp["x"],
            y=temp["y"],
            z=temp["z"],
            mode="markers",
            marker=dict(size=1, color=temp["cluster"], colorscale="hot", opacity=0.8),
        )
    ],
    layout=go.Layout(
        width=800,
        height=800,
        scene=dict(
            xaxis=dict(title="X", range=[min_global, max_global]),
            yaxis=dict(title="Y", range=[min_global, max_global]),
            zaxis=dict(title="Z", range=[min_global, max_global]),
        ),
    ),
)

fig.show()